<a href="https://colab.research.google.com/github/ohvictor/RN-TP2/blob/main/RN_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle: Rossmann store data

## 1. Data Visualization

### Métricas
La métrica de la competencia fue RMSPE para normalizar los errores de las predicciones. Al estar haciendo predicciones sobre el volumen de ventas, el cual puede ser grande o chico, usar solo RMSE no podría captar la verdadera magnitud del error. Por ejemplo, el mismo error absoluto sería pequeño frente a un gran volumen de ventas, pero grande frente a un volumen pequeño, y usando RMSPE se podrá detectar la magnitud de este error de predicción.

### Datasets

Se verificaron las dimensiones de todos los datos agregados, de forma que cada entrada de datos provistos por la competencia también tenga su correspondiente dato externo.

Se observa que además de los datos provistos por la competencia acerca de las tiendas, esta submisión decide tomar en cuenta factores meteorológicos, las tendencias semanales y la locación política de cada estado como factores de interés.

## 2. Preprocesamiento

### Missing Data
Esta sección identifica vacíos de información en los datasets y actúa acordemente. Esto lo hace identificando qué headers tienen una cantidad de elementos menores a la cantidad de entradas del dataframe.

+ "Open": Se consideró la tienda cerrada si la información falta en el día 7. Caso contrario se la consideró abierta.
+ "Competition Open Since Year", "Competition Open Since Month": Si la tienda no tiene esta información o es desde antes de "Enero 1990", se rellenó por defecto a "Enero 1990".
+ "Competition Distance": Si no hay datos, se considera que no hay competencia.
+ "Promo 2": Ante la falta de información se la considera que la tienda no participa en promociones.
+ "Weather": Ante la falta de datos se lo consideró un día de tiempo ideal.

### Fechas
Rellenados estos datos se cambió el formato de las fechas a tiempo desde una fecha de referencia. En este caso se tomó el día 15, al no ser este relevante para el entrenamiento del modelo.

### Tiempo + Estado
Se unieron también los datos meteorólogicos y de ubicación política como un solo dato externo.

### Googletrend
Prepara los datos para poder ser indexables por estados, y agrega las abreviaciones alternativas de un estado que no aparecía en el Googletrend. Esto permite buscar las tendencias por estado y fecha para poder ser unidas al dataset de entrenamiento.

También se separan las tendencias globales de Alemania de las tendencias locales de cada estado.

### Unión de Datos
Finalmente, se unieron todos los datos: los provistos por la competencia más los datos externos como el tiempo meteorológicos y la ubicación política de las tiendas. Además se cambió el formato de las fechas a "Días participados en la competencia" y "Días de promoción".

Cualquier dato de "Días de participación en la competencia" se lo considera 0 si es negativo. Además, se limita la participación a la competencia a 2 años máximo, y el tiempo participado se lo convierte a cantidad de meses.

## 3. Preprocesamiento de Duraciones

#### Paso StateHoliday a boolean

No es de interés saber específicamente qué feriado se lleva a cabo, solo si es un feriado, por lo tanto se convirtió esta información a formato booleano. 

#### Duraciones
Se extraen datos para explicar las relaciones que hay de tiempo. Sea asi, tiempo entre eventos y promedios correntes. En general el procedimiento se realiza entre filas, pero como se realiza entre columnas se define *elapsed*. En este, se hace un recuento acumulativo para un marco de datos ordenado, rastreando la ultima aparicion del campo, al momento de encontrarlo se resetea a cero y comienza de nuevo el recuento. 

Como se quiere observar en distintos casos y en el caso que sea antes o despues se utiliza una funcion que aplica *elapsed* a las distintas columnas del *DataFrame*. 

Por ejemplo, para la realizacion en el caso de *'After SchoolHoliday'* se ordena por *Store* y por *Date* y se tiene el tiempo transcurrido entre el dato actual y el anterior.

Los procedimientos se realizan tanto en el *DataFrame* de entrenamiento y de testeo. 

Cabe mencionar, que puede haber errores en el procedimiento por lo que aparecen *NaT (Not a Time)* que se deben corregir. En este caso, se pone el valor cero.

En resumen, esta sección calcula el tiempo transcurrido desde el último evento y el tiempo transcurrido hasta el siguiente evento (lo cual entrega un valor negativo, dado que apunta a un evento futuro). Los eventos que analiza son feriados escolares, estatales y promociones.

En el gráfico superior se puede observar la cantidad de días faltantes hasta el próximo evento correspondiente, mientras que en el gráfico inferior los días transcurridos desde el último evento correspondiente.

#### "Suavizado" de columnas temporales

El procedimiento consta de ordenar por fecha (*sort_index()*), posterior conteo del número de eventos de interés (*sum()*) definidos en columnas en la semana siguiente (rolling ()), agrupados por Store (groupby()). Se ordena por fecha tambien con el comando *sort_index(ascending = False)* para ordenar al reves. Cuenta cuantas va a haber en una semana de atras para adelante y de adelante para atras. 

Posteriormente, se eliminan los indices de las tiendas agrupados en la ventana establecida por *rolling()*.

El resultado final es una línea temporal de la cantidad de feriados futuros y pasados, y la cantidad de promociones futuras y pasadas para cada tienda.


## 4. Normalización y Encodificación

### Variables Categoricas
Toma las variables categoricas de los datos preprocesados anteriormente que se encuentran en el *DataFrame* tanto de entrenamiento como de testeo. Luego se obseva el tipo de variables, cuántos valores distintos tiene y si tienen *missing data*. Si el preprocesamiento fue correcto no deberia haber *missing data*.

### Variables Continuas
Toma las variables continuas de los datos preprocesados anteriormente que se encuentran en el *DataFrame* tanto de entrenamiento como de testeo. Luego se obseva el tipo de variables, cuantos valores distintos tiene y si tienen *missing data*. Si el preprocesamiento fue correcto no deberia haber *missing data*.

### Normalizacion en variables contunuas por *StandardScaler()* y *LabelEncode()* en categoricas. 

* *LabelEncode()*: codifica las etiquetas dentro de 0 y n-1, se realiza para las variables categoricas mencionadas anteriormente.  
* *StandardScaler()*: Estandarizacion de las características eliminando la media y escalando a la varianza de la unidad para las variables continuas
* *DataFrameMapper*: permite agrupar y almacenar todos los pasos del preprocesamiento de datos en un solo objeto, y aplicarlos a cualquier conjunto de datos con una sola operación.
  * mapper_cat para categoricas 
  * mapper_cont para continuas 

Se aplican tanto en el *DataFrame* de entrenamiento como de testeo.

Finalmente, al utilizar DataFrameSummary podemos observar que las variables que aparecian como *'categorical'* ahora aparecen como *'numeric'*.



Se divide inicialmetne el *DataFrame* entre un *DataFrame* de entrenamiento y otro de validacion, observandose cuantos datos quedan en cada uno. Se elige dividir a partir del 01/07/2015. 

Se define la funcion para el error cuadratico medio (RMSPE), el cual mide la cantidad de error que hay entre dos conjuntos de datos. En otras palabras, compara un valor predicho y un valor observado o conocido. Los valores más bajos de RMSPE indican un mejor ajuste. RMSPE es una buena medida de la precisión con que el modelo predice la respuesta, y es el criterio más importante para ajustar si el propósito principal del modelo es la predicción. Se utiliza el mismo para observar como se comportan las variables de interes contra su media por ejemplo en el caso de *Sales* normalizada por el maximo.

En la funcion *get_mean_by_column* se agrupan los valores segun lo establecido en la columna y se observan como se modifica el error en los distintos casos. A su vez, se obtienen las metricas de la funcion *get_metric*. Se observa en todos los casos que para Train el valor del RSME es mayor. Ademas, agrupar por *'StateHoliday'* hace que el mismo sea mayor y por *'Store'* se obtiene el menor. Siempre se utiliza *Sales_norm* (*Sales* normalizada por el maximo).

Nota: En el notebook 4.1 marca la fórmula como RMSE, pero es RMSPE como se definió el el notebook 1.0. Sin embargo, como las ventas ya fueron normalizadas, terminan teniendo el mismo propósito.

Se observaron luego las ventas normalizadas promedio agrupando por diferentes categorías. Agrupando no por tienda, se obtienen valores mayores de esta media, en ambos los datos de entrenamiento y validación.

### Estructura de la Red
Una vez codificadas las categorías y normalizados los parámetros de la red, esta fue construida en keras.

## 5. Embeddings
Como discutido en clase, el propósito de los embedding es la inclusión de datos que no serán utilizados luego para la predicción a fin de evitar el efecto de overfitting si solo se entrenara al modelo con los datos de entrada iniciales de la competencia.

Inicialmente se vuelven a observar las distintas tablas para entrar en tema y se hace el *Summary* de las variables categóricas. 
### Asignación de dimensión de embeddings
Para la asignación de dimensión de embeddings se utilizan las variables categóricas en forma de diccionario. Las variables que se utilicen y sean continuas no necesitan *embeddings*. 
##### Ejemplo con *’DayOfWeek’*, *’Store’* y *’CompetitionDistance’*
Se establecen *embeddings* para las variables *’Store’* y *'DayOfWeek'* ya que la variable *’CompetitionDistance’* no lo necesita por ser una variable continua. Luego, se aplanan las matrices a través de la función *Flaten()*. Una vez que se tienen aplanadas, entonces se concatenan las 3. 
Con la instrucción Dense se agrega una capa oculta a la red neuronal, la primera tiene 1000 nodos y la segunda 500. Y como salida tenemos activacion lineal y 1 nodo. Se utilizan dos capas densas con activaciones de RELU. 
Una vez que se obtuvieron las capas densas se plantea el modelo con la función de **tensorflow.keras.models** donde se ingresan los inputs y el output; y se observa el modelo creado. En el *Summary* se puede ver que se entrenan mas de 600 mil parámetros. 
Se define que a partir del 1/7/2015 se toma el corte para los dataframes de validación y de entrenamiento (ya explicado anteriormente). Se observa el porcentaje del entrenamiento en relación al total de datos. Se divide la matriz para X de entrenamiento y de validación con la función de numpy, hsplit. 

Ahora sólo queda entrenar, para ello, se realiza lo siguiente:
1.	Normalizamos y a partir de la media y el desvio estándar. Tanto y de entrenamiento como y de validación. 
2.	Le decimos al modelo que vamos a usar para entrenar y que utilizamos para validar. La cantidad de *batches* puede variar (inicialmente se observa con 256) y la cantidad de vueltas completas o *epochs* (inicialmente 1)
3.	El resultado del entrenamiento se guarda en la variable history. 

## 6. Full Model

Se habilita el uso creciente de la GPU, se podría cambiar a un porcentaje pero en este caso por ahí no es conveniente.
Se repiten los pasos iniciales del 5. 
Cuando se define cat_var_dict se observa que algunos parámetros son distintos.  Además, cat_vars = ['Store', 'DayOfWeek'] y contin_vars = ['BeforeStateHoliday_bool', 'Max_TemperatureC'], a diferencia de lo que se establece en el 5. 
Las funciones en [32] es lo mismo que se hace en la parte 5 pero definido como funciones, teniendo en cuenta si son continuas o categóricas. 
Se puede observar que en las capas densas se establece un regularizador para aplicar una penalización en el kernel de la capa. El tipo de activación que se mantiene es ‘ReLu’. 

Línea [37]: Usando model.summary() podemos ver una descripción general de la arquitectura del modelo. La capa de entrada acepta un tensor de forma (Ninguno, 1), lo que significa que cada muestra debe transformarse en un vector de 1 elemento. Se puede ver que la cantidad de parámetros siguie siendo mayor a 600 mil. 

En la línea [46] se guardan los pesos del modelo entrenado, la arquitectura del modelo se reproducirá usando el código y finalmente los pesos se cargarán en esa arquitectura. 
Por otro lado, se utiliza la función compile() que es método para especificar una pérdida, las métricas, y un optimizador. En este caso se esta utilizando el optimizador Adam(), las métricas ‘mse’ y rmspe y como perdida ‘mse’. El optimizado Adam() es un método de descenso de gradiente estocástico que se basa en la estimación adaptativa de momentos de primer y segundo orden. Se estableció lr = 0.001 que es el predeterminado.

Finalmente, se podría iterar entre combinaciones de embeddings para observar el comportamiento semántico de cada dato y así ver cuáles realmente generar una diferencia en el modelo y que ayuden contra el overfitting.

Luego de establecer el modelo que se va a utilizar se realiza model.fit() con los parámetros anteriormente explicados. Este model.fit() se guarda en una variable *history* de donde vamos a poder extraer distintas métricas y graficarlas para poder observar los cambios que ocurrieron. En este caso se graficaron *’val_loss’* (*Validation Loss*)y *’loss’* (*Training Loss*); donde xlabel es *"Num of Epochs"* e ylabel es *"Loss"*. 

### Baseline
Ilustra el caso base de una submisión de prueba con las correspondientes ventas promedio.

### Análisis de Embedding
- Día de la semana: presentan comportamientos similares los días martes, miércoles, jueves y viernes, mientras que lunes, sábado y domingo se comportan de maneras semánticamente distintas, alejadas del grupo anterior y de sí mismas.

- Mes: Se presenta una fuerte concentración del mes de enero, abril, junio, julio y agosto. En contraste, todos los otros meses tienen comportamientos muy distintos respecto a este grupo y entre sí mismos.